《[利用Python进行数据分析](https://book.douban.com/subject/25779298/)》读书笔记。
 
 [第10章](/2017/07/20/python_data_analysis10.html)  第9节：性能和内存使用方面的注意事项

所有用到的数据可以从[作者的 github](https://github.com/wesm/pydata-book)下载。


In [1]:
%pylab inline
import pandas as pd
from datetime import datetime
from pandas import Series, DataFrame

Populating the interactive namespace from numpy and matplotlib



TimeSeries和Period都是以64位整数表示的（即NumPy的datetime64数据类型）。

也就是说，对于每个数据点，其时间戳需要占用8字节内存。

因此，含有一百万个float64数据点的时间序列需要占用大约16MB的内存空间。由于pandas会尽量在多个时间序列之间共享索引，所以创建现有时间序列的视图不会占用更多内存。

此外，低频率索引（日以上）会被存放在一个中心缓存中，所以任何固定频率的索引都是该日期缓存的视图。所以。如果你有一个很大的低频率时间序列，索引所占用的内存空间将不会很大。

性能方面，pandas对数据对齐（两个不同索引的ts1 + ts2的幕后工作）和重采样运算进行了高度优化。


In [3]:
# 下面这个例子将一亿个数据点聚合为OHLC
rng = pd.date_range('1/1/2000', periods=10000000, freq='10ms')
ts = Series(np.random.randn(len(rng)), index=rng)
ts.head()

2000-01-01 00:00:00.000    0.219509
2000-01-01 00:00:00.010    1.336106
2000-01-01 00:00:00.020   -0.197760
2000-01-01 00:00:00.030   -1.554178
2000-01-01 00:00:00.040    1.049711
Freq: 10L, dtype: float64

In [6]:
%timeit ts.resample('15min').ohlc()

1 loop, best of 3: 146 ms per loop


In [7]:
rng = pd.date_range('1/1/2000', periods=10000000, freq='1s')
ts = Series(np.random.randn(len(rng)), index=rng)
%timeit ts.resample('15s').ohlc()

1 loop, best of 3: 210 ms per loop
